
# Práctica de Aprendizaje Semi-Supervisado con CIFAR-100

En esta práctica se desarrollan distintos enfoques para el aprendizaje semi-supervisado sobre el conjunto de datos CIFAR-100. Se parte de un conjunto de 50.000 instancias de entrenamiento y 10.000 instancias de prueba (etiquetadas en 100 clases). Se procede a eliminar el 80% de las etiquetas en el conjunto de entrenamiento, obteniéndose:
- 10.000 instancias etiquetadas
- 40.000 instancias sin etiquetar

A continuación se detallan los ejercicios a desarrollar:
1. Entrenar un modelo (con al menos 4 capas densas y/o convolucionales) utilizando únicamente los datos etiquetados.
2. Entrenar el mismo modelo usando auto-aprendizaje (self-training) para incorporar los datos sin etiquetar.
3. Entrenar un modelo semi-supervisado tipo autoencoder en dos pasos: primero entrenar el autoencoder (utilizando la misma arquitectura encoder que en 1 y 2, salvo el último bloque) y luego entrenar el clasificador.
4. Entrenar un modelo semi-supervisado de tipo autoencoder en un único paso (reconstrucción y clasificación simultánea).
5. Repetir los entrenamientos anteriores eliminando aquellas instancias no etiquetadas atípicas (según la técnica explicada en el Notebook 5, usando un valor de 𝑣 = 0.9).
6. Repetir los Ejercicios 3–5 usando la técnica del apartado “Hay vida más allá del autoencoder” (manteniendo la misma arquitectura encoder).

## Preparación: Carga y separación de datos

Utilizaremos la utilidad de Keras para cargar el conjunto de datos CIFAR-100. Posteriormente se separará el conjunto de entrenamiento en:
- 10.000 instancias etiquetadas.
- 40.000 instancias sin etiqueta (se ignoran las etiquetas originales para el entrenamiento semi-supervisado).



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers

In [2]:

# Cargar CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# Normalización de imágenes
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Convertir etiquetas a one-hot para la clasificación
y_train_cat = to_categorical(y_train, 100)
y_test_cat  = to_categorical(y_test, 100)

# Definir número de datos etiquetados (10k) y sin etiquetar (40k)
n_labeled = 10000
n_total = x_train.shape[0]

# Mezclar aleatoriamente el conjunto de entrenamiento
indices = np.arange(n_total)
np.random.shuffle(indices)

# Seleccionar índices para etiquetado y sin etiquetar
labeled_indices = indices[:n_labeled]
unlabeled_indices = indices[n_labeled:]

x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train_cat[labeled_indices]

x_train_unlabeled = x_train[unlabeled_indices]
# Nota: Las etiquetas originales de x_train_unlabeled se ignoran para el auto-aprendizaje

print("Datos etiquetados:", x_train_labeled.shape, y_train_labeled.shape)
print("Datos sin etiquetar:", x_train_unlabeled.shape)

Datos etiquetados: (10000, 32, 32, 3) (10000, 100)
Datos sin etiquetar: (40000, 32, 32, 3)



## Ejercicio 1: Modelo supervisado utilizando únicamente los datos etiquetados

En este ejercicio se define y entrena un modelo de TensorFlow basado en una arquitectura que combina capas convolucionales y densas (al menos 4 bloques) utilizando solo los 10.000 ejemplos etiquetados.

### Preguntas a responder:
a. **¿Qué red has escogido? ¿Por qué? ¿Cómo la has entrenado?**  
   Se ha optado por una red CNN simple que consta de dos bloques convolucionales seguidos de capas densas. Se eligió esta arquitectura por su eficacia en tareas de clasificación de imágenes y su relativa simplicidad.

b. **¿Cuál es el rendimiento del modelo en entrenamiento? ¿Y en prueba?**  
   Se reportarán las métricas de pérdida y precisión tanto en el conjunto de entrenamiento como en el de prueba.

c. **Conclusiones**  
   Se analizarán las limitaciones al usar pocos datos etiquetados.



In [ ]:

def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        
        layers.Dense(256, activation='relu'),
        layers.Dense(100, activation='softmax')
    ])

    model.compile(optimizer=optimizers.Adam(), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


model_sup = build_model()


# Entrenamiento con solo datos etiquetados
history_sup = model_sup.fit(x_train_labeled, y_train_labeled,
                            epochs=20, batch_size=64,
                            validation_data=(x_test, y_test_cat),
                            verbose=2)

# Evaluación
train_loss, train_acc = model_sup.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss, test_acc   = model_sup.evaluate(x_test, y_test_cat, verbose=0)
print("Supervisado: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc, test_acc))


c:\Users\108057\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'NoneType' object has no attribute 'compile'

### Comentarios del Ejercicio 1

- **Arquitectura:** Se utilizó una red CNN con dos bloques convolucionales y una capa densa final para clasificación.
- **Entrenamiento:** Se entrenó con 20 épocas usando Adam y una función de pérdida de entropía cruzada.
- **Resultados:** Se muestran las métricas en entrenamiento y prueba. Probablemente se observe un rendimiento moderado debido a la cantidad reducida de datos etiquetados.



## Ejercicio 2: Auto-aprendizaje (Self-Training)

En este ejercicio se reutiliza el mismo modelo definido en el Ejercicio 1, pero se incorporan las instancias sin etiquetar mediante la técnica de auto-aprendizaje. La idea es:
1. Entrenar el modelo inicialmente con los datos etiquetados.
2. Utilizar el modelo para predecir etiquetas en los datos sin etiquetar.
3. Seleccionar aquellas predicciones con alta certeza (por ejemplo, una probabilidad mayor a un umbral definido) y agregarlas al conjunto de entrenamiento, ponderando (opcionalmente) su contribución según la confianza.

### Preguntas a responder:
a. **¿Qué parámetros has definido para el entrenamiento?**  
   Se define un umbral de confianza (por ejemplo, 0.9) para aceptar pseudo-etiquetas.

b. **Rendimiento en entrenamiento y prueba.**

c. **¿Se mejoran los resultados respecto al Ejercicio 1?**

d. **Conclusiones sobre los resultados.**


In [ ]:
# Umbral para la certeza de las pseudo-etiquetas
confidence_threshold = 0.9

# Realizar predicciones en los datos sin etiquetar
pseudo_labels_prob = model_sup.predict(x_train_unlabeled)
pseudo_labels = np.argmax(pseudo_labels_prob, axis=1)
pseudo_labels_cat = to_categorical(pseudo_labels, 100)

# Seleccionar aquellos ejemplos con alta confianza
max_probs = np.max(pseudo_labels_prob, axis=1)
selected = max_probs >= confidence_threshold

print("Número de pseudo-etiquetas seleccionadas:", np.sum(selected))

# Combinar datos etiquetados con pseudo-etiquetados de alta confianza
x_combined = np.concatenate([x_train_labeled, x_train_unlabeled[selected]], axis=0)
y_combined = np.concatenate([y_train_labeled, pseudo_labels_cat[selected]], axis=0)

# Reiniciar el modelo (o seguir afinando)
model_self = build_supervised_model()
model_self.compile(optimizer=optimizers.Adam(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

history_self = model_self.fit(x_combined, y_combined,
                              epochs=20, batch_size=64,
                              validation_data=(x_test, y_test_cat),
                              verbose=2)

train_loss_self, train_acc_self = model_self.evaluate(x_combined, y_combined, verbose=0)
test_loss_self, test_acc_self   = model_self.evaluate(x_test, y_test_cat, verbose=0)
print("Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc_self, test_acc_self))


### Comentarios del Ejercicio 2

- **Parámetros:** Se ha definido un umbral de confianza de 0.9 para la selección de pseudo-etiquetas.
- **Resultados:** Se evalúa el rendimiento en entrenamiento y prueba del modelo entrenado con datos combinados.  
- **Comparación:** Se discute si el auto-aprendizaje mejora o no los resultados obtenidos en el Ejercicio 1, teniendo en cuenta que se incorpora información extra con cierto nivel de incertidumbre.




## Ejercicio 3: Autoencoder en dos pasos

Este ejercicio se realiza en dos etapas:
1. **Entrenamiento del autoencoder:** Se entrena un autoencoder que utiliza como encoder la parte convolucional (igual que la definida en los ejercicios anteriores, salvo el último bloque) para aprender una representación compacta de las imágenes.
2. **Entrenamiento del clasificador:** Se utiliza el encoder preentrenado, se congela y se añade una cabeza de clasificación (capas densas) que se entrena con los datos etiquetados.

### Preguntas a responder:
a. **Arquitectura y hiperparámetros.**

b. **Rendimiento en entrenamiento y prueba.**

c. **Comparación con los Ejercicios 1 y 2.**

d. **Conclusiones.**


In [ ]:
# Construir el autoencoder
def build_autoencoder():
    input_img = layers.Input(shape=x_train.shape[1:])
    # Encoder: se reutiliza parte de la arquitectura de Ej.1
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2,2), padding='same')(x)
    
    # Decoder: arquitectura simétrica
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2,2))(x)
    decoded = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)
    
    autoencoder = models.Model(input_img, decoded)
    encoder = models.Model(input_img, encoded)
    return autoencoder, encoder

autoencoder, encoder = build_autoencoder()
autoencoder.compile(optimizer='adam', loss='mse')

# Entrenar el autoencoder con todas las imágenes de entrenamiento (se puede usar tanto etiquetadas como sin etiquetar)
x_autoencoder = x_train  # Uso completo del entrenamiento sin distinguir etiqueta
autoencoder.fit(x_autoencoder, x_autoencoder,
                epochs=20, batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                verbose=2)

# %% 
# Ahora, se congela el encoder y se añade una cabeza de clasificación
for layer in encoder.layers:
    layer.trainable = False

# Construir el clasificador utilizando el encoder
encoded_input = layers.Input(shape=encoder.output_shape[1:])
x = layers.Flatten()(encoded_input)
x = layers.Dense(256, activation='relu')(x)
output = layers.Dense(100, activation='softmax')(x)
classifier_head = models.Model(encoded_input, output)

# Conectar el encoder y la cabeza de clasificación
input_img = layers.Input(shape=x_train.shape[1:])
features = encoder(input_img)
classifier_output = classifier_head(features)
model_autoencoder_classifier = models.Model(input_img, classifier_output)

model_autoencoder_classifier.compile(optimizer=optimizers.Adam(),
                                     loss='categorical_crossentropy',
                                     metrics=['accuracy'])

# Entrenar el clasificador con los datos etiquetados
history_ae = model_autoencoder_classifier.fit(x_train_labeled, y_train_labeled,
                                              epochs=20, batch_size=64,
                                              validation_data=(x_test, y_test_cat),
                                              verbose=2)

train_loss_ae, train_acc_ae = model_autoencoder_classifier.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss_ae, test_acc_ae   = model_autoencoder_classifier.evaluate(x_test, y_test_cat, verbose=0)
print("Autoencoder (2 pasos): Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc_ae, test_acc_ae))



### Comentarios del Ejercicio 3

- **Arquitectura:**  
  - *Autoencoder:* Se entrena un modelo que aprende a reconstruir las imágenes.  
  - *Encoder:* Es la parte convolucional preentrenada.  
  - *Clasificador:* Se añade una cabeza densa para la clasificación.
- **Hiperparámetros:** Se empleó MSE para la reconstrucción y entropía cruzada para la clasificación, con 20 épocas en cada fase.
- **Resultados:** Se evalúa el rendimiento en entrenamiento y prueba, y se compara con los enfoques anteriores.



## Ejercicio 4: Autoencoder con clasificación simultánea (una etapa)

En este ejercicio se entrena un modelo que combina la reconstrucción del autoencoder y la clasificación en un único entrenamiento. La arquitectura del encoder es la misma que en el Ejercicio 3 y la combinación encoder+clasificador es similar al del Ejercicio 1.

### Preguntas a responder:
a. **Arquitectura y hiperparámetros.**

b. **Rendimiento en entrenamiento y prueba.**

c. **¿Se mejoran los resultados?**

d. **Conclusiones.**


In [ ]:
# Definir un modelo multitarea: reconstrucción y clasificación
input_img = layers.Input(shape=x_train.shape[1:])

# Encoder (mismo que antes)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2,2), padding='same')(x)
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2,2), padding='same')(x)

# Rama de reconstrucción (decoder)
x_rec = layers.Conv2D(64, (3,3), activation='relu', padding='same')(encoded)
x_rec = layers.UpSampling2D((2,2))(x_rec)
x_rec = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x_rec)
x_rec = layers.UpSampling2D((2,2))(x_rec)
decoded = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same', name='decoded')(x_rec)

# Rama de clasificación (cabeza)
x_cls = layers.Flatten()(encoded)
x_cls = layers.Dense(512, activation='relu')(x_cls)
classification = layers.Dense(100, activation='softmax', name='classification')(x_cls)

model_multi = models.Model(input_img, [decoded, classification])

# Compilación con dos pérdidas: reconstrucción (MSE) y clasificación (entropía cruzada).
model_multi.compile(optimizer='adam',
                    loss={'decoded': 'mse', 'classification': 'categorical_crossentropy'},
                    loss_weights={'decoded': 0.5, 'classification': 1.0},
                    metrics={'classification': 'accuracy'})

# Entrenar el modelo multitarea: para la rama de reconstrucción se usan las imágenes de entrada
history_multi = model_multi.fit(x_train_labeled, {'decoded': x_train_labeled, 'classification': y_train_labeled},
                                epochs=20, batch_size=64,
                                validation_data=(x_test, {'decoded': x_test, 'classification': y_test_cat}),
                                verbose=2)

# Evaluar solo la parte clasificadora
results = model_multi.evaluate(x_test, {'decoded': x_test, 'classification': y_test_cat}, verbose=0)
print("Multi-tarea: Test Accuracy (clasificación): {:.4f}".format(results[3]))  # results[3] corresponde a la métrica de 'classification'



### Comentarios del Ejercicio 4

- **Arquitectura:** Se utiliza un modelo con dos salidas, una para la reconstrucción de la imagen y otra para la clasificación.
- **Hiperparámetros:** Se emplean dos pérdidas (ponderadas) y se entrena en 20 épocas.
- **Resultados:** Se analizan las métricas de clasificación y se comparan con los ejercicios anteriores.




## Ejercicio 5: Eliminación de instancias no etiquetadas atípicas

En este ejercicio se repiten los entrenamientos de los Ejercicios 1–4 pero se eliminan las instancias sin etiquetar consideradas atípicas en relación con los datos etiquetados. Se utiliza la técnica explicada (por ejemplo, seleccionando solo aquellas pseudo-etiquetas cuya confianza sea superior a 𝑣 = 0.9) y se utiliza la misma arquitectura de clasificación que en el Ejercicio 1, salvo la capa de salida.

### Pregunta a responder:
a. ¿Se mejoran los resultados con respecto a los ejercicios anteriores? ¿Qué conclusiones se sacan?


In [ ]:

# Ejemplo: Reaplicar el auto-aprendizaje del Ejercicio 2 con selección basada en v=0.9
v = 0.9  # umbral definido
max_probs = np.max(pseudo_labels_prob, axis=1)
selected_v = max_probs >= v
print("Instancias sin etiqueta seleccionadas (v=0.9):", np.sum(selected_v))

# Se combinan los datos etiquetados con los pseudo-etiquetados filtrados
x_combined_v = np.concatenate([x_train_labeled, x_train_unlabeled[selected_v]], axis=0)
y_combined_v = np.concatenate([y_train_labeled, pseudo_labels_cat[selected_v]], axis=0)

# Reiniciar y entrenar el modelo supervisado (con arquitectura similar a Ej.1)
model_filtered = build_supervised_model()
model_filtered.compile(optimizer=optimizers.Adam(),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

history_filtered = model_filtered.fit(x_combined_v, y_combined_v,
                                      epochs=20, batch_size=64,
                                      validation_data=(x_test, y_test_cat),
                                      verbose=2)

train_loss_f, train_acc_f = model_filtered.evaluate(x_combined_v, y_combined_v, verbose=0)
test_loss_f, test_acc_f   = model_filtered.evaluate(x_test, y_test_cat, verbose=0)
print("Filtered Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc_f, test_acc_f))

# %% [markdown]
"""
### Comentarios del Ejercicio 5

- Se ha aplicado la técnica de filtrado utilizando un umbral 𝑣 = 0.9 para eliminar los ejemplos sin etiqueta menos confiables.
- Se compara el rendimiento del modelo con el obtenido en ejercicios anteriores.
- Se discuten las mejoras y las limitaciones encontradas tras eliminar los datos atípicos.
"""

# %% [markdown]
"""
## Ejercicio 6: Uso de la técnica "Hay vida más allá del autoencoder"

En este ejercicio se repiten los entrenamientos de los Ejercicios 3–5 cambiando el autoencoder por la técnica definida en el apartado “Hay vida más allá del autoencoder”. La arquitectura de la red se mantiene igual que la parte encoder del autoencoder definido anteriormente, y se asegura que el modelo entrene correctamente.

### Preguntas a responder:
a. Arquitectura y hiperparámetros.
b. Rendimiento en entrenamiento y prueba.
c. Comparación con los ejercicios anteriores.
d. Conclusiones.
 
**Nota:** La implementación de esta técnica puede variar; a continuación se muestra una posible aproximación en la que se modifica el esquema de entrenamiento para obtener representaciones útiles para la clasificación sin reconstruir la imagen completa.
"""

# %% 
# Ejemplo: Construcción de un modelo basado en la técnica alternativa
def build_alternative_model():
    input_img = layers.Input(shape=x_train.shape[1:])
    # Encoder idéntico al anterior
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2,2), padding='same')(x)
    # Se entrena directamente para clasificación
    x_cls = layers.Flatten()(encoded)
    x_cls = layers.Dense(512, activation='relu')(x_cls)
    classification = layers.Dense(100, activation='softmax')(x_cls)
    model_alt = models.Model(input_img, classification)
    return model_alt

# Entrenar el modelo alternativo utilizando la técnica de filtrado (como en el Ej.5)
model_alt = build_alternative_model()
model_alt.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Se puede entrenar primero con los datos etiquetados y posteriormente incorporar los pseudo-datos filtrados
history_alt_1 = model_alt.fit(x_train_labeled, y_train_labeled,
                              epochs=20, batch_size=64,
                              validation_data=(x_test, y_test_cat),
                              verbose=2)

# Incorporar datos pseudo-etiquetados (filtrados con el umbral v=0.9)
history_alt_2 = model_alt.fit(x_train_unlabeled[selected_v], pseudo_labels_cat[selected_v],
                              epochs=10, batch_size=64,
                              validation_data=(x_test, y_test_cat),
                              verbose=2)

train_loss_alt, train_acc_alt = model_alt.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss_alt, test_acc_alt   = model_alt.evaluate(x_test, y_test_cat, verbose=0)
print("Alternative Model: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc_alt, test_acc_alt))

# %% [markdown]
"""
### Comentarios del Ejercicio 6

- **Arquitectura:** Se utiliza la parte encoder del autoencoder para extraer características, a las que se añade una cabeza de clasificación.
- **Hiperparámetros:** Se emplean las mismas configuraciones de optimización y pérdida que en los ejercicios anteriores.
- **Resultados y comparación:** Se evalúa el rendimiento y se compara con el resto de métodos implementados.
- **Conclusiones:** Se discuten las ventajas y desventajas de la técnica alternativa respecto al uso del autoencoder tradicional.

---

## Conclusión General

En este Notebook se ha demostrado cómo:
- El entrenamiento supervisado con pocos datos puede ser limitado.
- El auto-aprendizaje permite incorporar datos sin etiquetar y mejorar potencialmente el rendimiento.
- Los modelos basados en autoencoder (tanto en dos pasos como en un paso) y técnicas alternativas pueden extraer representaciones robustas para mejorar la clasificación.
- El filtrado de ejemplos atípicos (según un umbral de confianza) puede ayudar a evitar el “ruido” de pseudo-etiquetas poco confiables.

Cada uno de estos enfoques debe evaluarse en función de las necesidades y limitaciones del problema concreto.

> Nota: Los hiperparámetros, número de épocas y umbrales utilizados son ejemplos. En un entorno real, se recomienda realizar un tuning exhaustivo.

Este Notebook ofrece un esqueleto que puede adaptarse y ampliarse según las necesidades de la práctica.
"""
